In [ ]:
# Name - Raj Mahajan
# Roll Number - CS22M067
# Assignment 1

In [ ]:
!pip install wandb
import wandb
wandb.login(key = "c28eda24dacbd50a57332e445cfdfb2f4c16799c")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 KB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 3.5 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=19b9448e374ea3f613f30ee181bbe22bb050fdd2138dcc8a2a418c396bf87e75
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import wandb
wandb.init(project="DL_A1_optimizers_combined_14.ipynb")

wandb: Currently logged in as: rajmahajan24. Use `wandb login --relogin` to force relogin


In [ ]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split

In [ ]:
# Load the Fashion-MNIST dataset
(X_train, Y_train), (x_test, y_test) = fashion_mnist.load_data()
X_train=X_train.reshape(X_train.shape[0],784)
x_test=x_test.reshape(x_test.shape[0],784)
Y_train=Y_train.reshape(60000,1)
y_test=y_test.reshape(10000,1)
# X_train.shape
Y_train.shape

4422102/4422102 [==============================] - 0s 0us/step


(60000, 1)

In [ ]:
# Normalizing the data
X_train=X_train/255.0
x_test=x_test/255.0
y_test.shape

(10000, 1)

In [ ]:
x_train,x_val,y_train,y_val = train_test_split(X_train , Y_train , test_size = 0.1 , random_state = 100)
# x_train.shape
x_val.shape

(6000, 784)

In [ ]:
def actual_prob_dist(y) :
  ans=np.zeros((10, 1))
  ans[y]=1
  return(ans)

def sigmoid(a) :
  # s=1/(1+np.exp(a))

  if isinstance(a,np.ndarray):
    return 1.0 / (1.0 + np.exp(-a))
  else :
    if (x>=0):
      return 1.0 / (1.0 + np.exp(-a))
    else :
      return np.exp(a) / ( 1.0 + np.exp(a))
  # return(s)



def derivative_sigmoid(x):
  ans = sigmoid(x) * (1 -sigmoid(x))
  return(ans)

def relu(x):
    return np.maximum(0,x)

def derivative_relu(x):
    return 1*(x>0)

def tanh(x):
    return np.tanh(x)

def derivative_tanh(x):
    return (1 - (np.tanh(x)**2))

def softmax(v):
  # ans = np.exp(v) / np.sum(np.exp(v), axis=0)
  z = v - max(v)
  numerator = np.exp(z)
  denominator = np.sum(numerator)
  ans = numerator/denominator
  return(ans)


def derivative_softmax(x):
  ans= softmax(x) * (1-softmax(x))
  return(ans)


def cross_entropy(y, yhat, batch_size):
   loss = (-1.0 * np.sum(np.multiply(y, np.log(yhat))))/batch_size
   return(loss)

In [ ]:
def add_list_grad(list_grad_x , list_grad_y):
  if(len(list_grad_x) == 0 ):
    return(list_grad_y)
  for i in range(0,len(list_grad_x)):
    list_grad_x[i] += list_grad_y[i]
  return(list_grad_x)

def add_list_grad_eta(list_grad_x , list_grad_y, eta):
  if(len(list_grad_x) == 0 ):
    return(list_grad_y)
  for i in range(0,len(list_grad_x)):
    list_grad_x[i] -= eta * list_grad_y[i].transpose()
  return(list_grad_x)

def initilize_theta_random_normal(num_layers,neurons):
  theta=[]
  i=0
  temp=np.random.randn(neurons[0], neurons[1])
  theta.append(temp)
  while i< len(neurons)-3 :
    temp=np.random.randn(neurons[i+1], neurons[i+2])
    theta.append(temp)
    i+=1
  temp=np.random.randn(neurons[-2], neurons[-1])
  theta.append(temp)
  return(theta)

def initilize_theta_random_uniform(num_layers,neurons):
  theta=[]
  i=0
  temp=np.random.rand(neurons[0], neurons[1])
  theta.append(temp)
  while i< len(neurons)-3 :
    temp=np.random.rand(neurons[i+1], neurons[i+2])
    theta.append(temp)
    i+=1
  temp=np.random.rand(neurons[-2], neurons[-1])
  theta.append(temp)
  return(theta)

def initilize_theta_xavier(num_layers,neurons):
  theta=[]
  i=0
  temp=np.random.randn(neurons[0], neurons[1]) * np.sqrt(2/(neurons[0]  + neurons[1])) # xavier
  theta.append(temp)
  while i< len(neurons)-3 :
    temp=np.random.randn(neurons[i+1], neurons[i+2]) * np.sqrt(2/(neurons[i+1] + neurons[i+2])) # xavier
    theta.append(temp)
    i+=1
  temp=np.random.randn(neurons[-2], neurons[-1]) * np.sqrt(2/(neurons[-2] + neurons[-1])) # xavier
  theta.append(temp)
  return(theta)

def initilize_bias(num_layers,neurons):
  b=[]
  i=0
  while i < num_layers :
    temp=np.zeros((1,neurons[i+1]))
    b.append(temp)
    i+=1
  temp=np.zeros((1, neurons[-1]))
  b.append(temp)
  return(b)




In [ ]:
def activation_fun(activation,a):
  if(activation == "sigmoid"):
    return(sigmoid(a))
  elif(activation == "relu"):
    return(relu(a))
  elif(activation == "tanh"):
    return(tanh(a))

def derivative_activation_fun(activation,a):
  if(activation == "sigmoid"):
    return(derivative_sigmoid(a))
  elif(activation == "relu"):
    return(derivative_relu(a))
  elif(activation == "tanh"):
    return(derivative_tanh(a))
  
def loss_function(loss_fun,yhat,y):
  if(loss_fun == "cross_entropy"):
    loss = (-math.log(yhat[y]))
  elif(loss_fun == "mse"):
    Y = actual_prob_dist(y)
    loss = (1/2) * np.sum((Y-yhat)**2)
  return(loss)


In [ ]:
# Constructed only for 1 image 
# h will be image input if 1d vector of length 28x28
def forward_prop(activation,theta,b,num_layers,h):
  h_list=[]
  h_list.append(h)
  a_list=[]
  for k in range(0,num_layers):
    a = b[k].transpose() + np.matmul(theta[k].transpose() , h)
    a_list.append(a)
    h= activation_fun(activation,a)
    h_list.append(h)
  # For last output layer
  a = b[num_layers].transpose() + np.matmul(theta[num_layers].transpose() , h)
  a_list.append(a)
  yhat = softmax(a)
  return(yhat,h_list,a_list)


In [ ]:
# only for 1 image
def backward_prop(theta,h_list,a_list,y,yhat,num_layers,batch_size,activation,loss_fun):
  grad_a=-(actual_prob_dist(y) - yhat)
  if(loss_fun == "mse"):
      grad_a *= derivative_softmax(a_list[len(a_list)-1]);
  list_grad_w=[]
  list_grad_b=[]
  for k in range(num_layers , -1,-1) :
    grad_w = np.matmul(grad_a , h_list[k].transpose()) 
    list_grad_w.append(grad_w / batch_size)
    grad_b = grad_a.copy()
    list_grad_b.append(grad_b / batch_size)
    if k >0:
      grad_h = np.matmul(theta[k] , grad_a)
      grad_a = grad_h * derivative_activation_fun(activation,a_list[k-1])
  return(list_grad_w , list_grad_b)
  

# grad_b.shape = 32 x 1
# grad_w.shape = 32 x 32
# grad_a.shape = 10/32 x 1

In [ ]:
def image_pridiction(x_test, y_test, theta, b, num_layers,activation):
  img_no = 0
  wrong_count = 0
  while(img_no<len(x_test)):
    img = x_test[img_no].reshape(784,1)
    clas = y_test[img_no]
    temp_ans = forward_prop(activation,theta,b,num_layers,img)
    yhat = temp_ans[0]
    temp = -1
    predicted_class = -1
    sum=0
    for i in range(0 , 10):
      sum+=yhat[i]
      if(temp < yhat[i]):
        predicted_class = i
        temp = yhat[i]
    if(clas != predicted_class):
      wrong_count+=1
    img_no+=1
  accuracy = (len(x_test) - wrong_count) * 100 / len(x_test)
  return(accuracy)

In [ ]:
def SGD(theta,b,epochs,eta,beta,activation,x_train,y_train,neurons, num_layers,batch_size,loss_fun,weight_decay):
  t=0
  global x_val
  global y_val
  while t<epochs :
    t += 1
    print(t)
    image_no = 0
    list_grad_w = []
    list_grad_b = []
    loss = 0
    val_loss = 0
    while(image_no<len(x_train)):
         
      # forward propgation
      h=x_train[image_no].reshape(784,1)
      y_hat,h_list,a_list = forward_prop(activation,theta,b,num_layers,h)
      # training loss
      y = y_train[image_no]
      if( y_hat[y]!= 0):
        loss += loss_function(loss_fun,y_hat,y) / len(x_train)
      # validation loss
      if(image_no < len(x_val)):
        h_v = x_val[image_no].reshape(784,1)
        y_hat_v,h_list_v,a_list_v = forward_prop(activation,theta,b,num_layers,h_v)
        y_v = y_val[image_no]
        if(y_hat_v[y_v] != 0):
          val_loss += loss_function(loss_fun,y_hat_v,y_v) / len(x_val)

      # back propogation
      bp_list = backward_prop(theta,h_list,a_list,y_train[image_no],y_hat,num_layers,batch_size,activation,loss_fun)
      list_grad_w = add_list_grad(list_grad_w.copy(), bp_list[0])
      list_grad_b = add_list_grad(list_grad_b.copy() , bp_list[1])
      if((image_no + 1) % batch_size == 0):
        list_grad_w.reverse()
        list_grad_b.reverse()
        for j in range(len(list_grad_w)):
          list_grad_w[j] += weight_decay * theta[j].transpose();
        theta = add_list_grad_eta(theta.copy() , list_grad_w, eta)
        b = add_list_grad_eta(b.copy(), list_grad_b , eta)
        list_grad_w.clear()
        list_grad_b.clear()
      # print(theta)
      image_no += 1

    # Update theta for last few images
    if (len(x_train) % batch_size != 0):
      list_grad_w.reverse()
      list_grad_b.reverse()
      for j in range(len(list_grad_w)):
          list_grad_w[j] += weight_decay * theta[j].transpose();
      theta = add_list_grad_eta(theta.copy() , list_grad_w, eta)
      b = add_list_grad_eta(b.copy(), list_grad_b , eta)

    for i in range(len(theta)):
        temp = (weight_decay / 2) * np.sum(np.square(theta[i]))
        loss += temp / len(x_train)
        val_loss += temp / len(x_val)
    print("training loss = ",loss)
    print("validation loss = ",val_loss)
    train_accuracy = image_pridiction(x_train,y_train,theta,b,num_layers,activation)
    print("trianing accuracy = ",train_accuracy)
    val_accuracy = image_pridiction(x_val,y_val,theta,b,num_layers,activation)
    print("validation_accuracy = ",val_accuracy)
    wandb.log({"train_accuracy": train_accuracy, "validation_accuracy": val_accuracy, "training_loss": loss, "validation cost": val_loss, 'epoch': t})

  return(theta,b)

In [ ]:
def momentum_SGD(theta,b,epochs,eta,beta,activation,x_train,y_train,neurons, num_layers,batch_size,loss_fun,weight_decay):

  t=0
  global x_val
  global y_val
  while t < epochs :
    t += 1
    print(t)
    image_no = 0
    list_u_grad_w = []
    list_u_grad_b = []
    list_grad_w = []
    list_grad_b = []
    prev_grad_uw = [] # storing previous point
    prev_grad_ub = [] # storing previous point
    loss = 0
    val_loss = 0
    while(image_no<len(x_train)):
      # forward propgation
      h=x_train[image_no].reshape(784,1)
      fp_list = forward_prop(activation,theta,b,num_layers,h)
      # backward propogation
      a_list = fp_list[2]
      h_list = fp_list[1]
      y_hat = fp_list[0]
      y = y_train[image_no]
      if( y_hat[y]!= 0):
        loss += loss_function(loss_fun,y_hat,y) / len(x_train)
      # validation loss
      if(image_no < len(x_val)):
        h_v = x_val[image_no].reshape(784,1)
        y_hat_v,h_list_v,a_list_v = forward_prop(activation,theta,b,num_layers,h_v)
        y_v = y_val[image_no]
        if(y_hat_v[y_v] != 0):
          val_loss += loss_function(loss_fun,y_hat_v,y_v) / len(x_val)

      # back propogation

      bp_list = backward_prop(theta,h_list,a_list,y_train[image_no],y_hat,num_layers,batch_size,activation,loss_fun)
      list_grad_w = add_list_grad(list_grad_w.copy(), bp_list[0])
      list_grad_b = add_list_grad(list_grad_b.copy() , bp_list[1])

      if((image_no+1) % batch_size == 0):
        list_grad_w.reverse()
        list_grad_b.reverse()
        for j in range(len(list_grad_w)):
          list_grad_w[j] += weight_decay * theta[j].transpose();
        
        if(len(prev_grad_uw) == 0):
          for i in range(len(list_grad_w)):
            prev_grad_uw.append(beta * list_grad_w[i])
            prev_grad_ub.append(beta * list_grad_b[i])

        for i in range(len(list_grad_w)):
          list_u_grad_w.append(beta * prev_grad_uw[i] + (1 - beta) * list_grad_w[i])
          list_u_grad_b.append(beta * prev_grad_ub[i] + (1 - beta) * list_grad_b[i])
        
        theta = add_list_grad_eta(theta.copy() , list_u_grad_w, eta)
        b = add_list_grad_eta(b.copy(), list_u_grad_b , eta)
        
        prev_grad_uw = list_u_grad_w.copy()
        prev_grad_ub = list_u_grad_b.copy()
        list_u_grad_w.clear()
        list_u_grad_b.clear()
        list_grad_w.clear()
        list_grad_b.clear()
      image_no += 1

    # Update theta for last few images
    if (len(x_train) % batch_size != 0):
      list_grad_w.reverse()
      list_grad_b.reverse()
      for j in range(len(list_grad_w)):
          list_grad_w[j] += weight_decay * theta[j].transpose();
      if(len(prev_grad_uw) == 0):
        for i in range(len(list_grad_w)):
          prev_grad_uw.append(beta * list_grad_w[i])
          prev_grad_ub.append(beta * list_grad_b[i])

      for i in range(len(list_grad_w)):
        list_u_grad_w.append(beta * prev_grad_uw[i] + (1 - beta) * list_grad_w[i])
        list_u_grad_b.append(beta * prev_grad_ub[i] + (1 - beta) * list_grad_b[i])
      
      theta = add_list_grad_eta(theta.copy() , list_u_grad_w, eta)
      b = add_list_grad_eta(b.copy(), list_u_grad_b , eta)
      
      prev_grad_uw = list_u_grad_w.copy()
      prev_grad_ub = list_u_grad_b.copy()
      list_u_grad_w.clear()
      list_u_grad_b.clear()
      list_grad_w.clear()
      list_grad_b.clear()

    for i in range(len(theta)):
        temp = (weight_decay / 2) * np.sum(np.square(theta[i]))
        loss += temp / len(x_train)
        val_loss += temp / len(x_val)
    print("training loss = ",loss)
    print("validation loss = ",val_loss)
    train_accuracy = image_pridiction(x_train,y_train,theta,b,num_layers,activation)
    print("trianing accuracy = ",train_accuracy)
    val_accuracy = image_pridiction(x_val,y_val,theta,b,num_layers,activation)
    print("validation_accuracy = ",val_accuracy)
    wandb.log({"train_accuracy": train_accuracy, "validation_accuracy": val_accuracy, "training_loss": loss, "validation cost": val_loss, 'epoch': t})
  return(theta,b)

In [ ]:
def RMS_SGD(theta,b,epochs,eta,beta,activation,x_train,y_train,neurons, num_layers,batch_size,loss_fun,weight_decay):
 
  t=0
  global x_val
  global y_val
  while t < epochs :
    t += 1
    print(t)
    image_no = 0
    list_u_grad_w = []
    list_u_grad_b = []
    list_grad_w = []
    list_grad_b = []
    prev_grad_uw = [] # storing previous point
    prev_grad_ub = [] # storing previous point
    loss = 0
    val_loss = 0
    while(image_no<len(x_train)):
      # forward propgation
      h=x_train[image_no].reshape(784,1)
      fp_list = forward_prop(activation,theta,b,num_layers,h)
      # backward propogation
      a_list = fp_list[2]
      h_list = fp_list[1]
      y_hat = fp_list[0]
      y = y_train[image_no]
      if( y_hat[y]!= 0):
        loss += loss_function(loss_fun,y_hat,y) / len(x_train) 
      # validation loss
      if(image_no < len(x_val)):
        h_v = x_val[image_no].reshape(784,1)
        y_hat_v,h_list_v,a_list_v = forward_prop(activation,theta,b,num_layers,h_v)
        y_v = y_val[image_no]
        if(y_hat_v[y_v] != 0):
          val_loss += loss_function(loss_fun,y_hat_v,y_v) / len(x_val)

      # back propogation
      bp_list = backward_prop(theta,h_list,a_list,y_train[image_no],y_hat,num_layers ,batch_size,activation,loss_fun)
      list_grad_w=add_list_grad(list_grad_w.copy(), bp_list[0])
      list_grad_b=add_list_grad(list_grad_b.copy() , bp_list[1])

      if((image_no + 1) % batch_size == 0):
        list_grad_w.reverse()
        list_grad_b.reverse()
        for j in range(len(list_grad_w)):
          list_grad_w[j] += weight_decay * theta[j].transpose();
        if(len(prev_grad_uw) == 0):
          for i in range(len(list_grad_w)):
            list_u_grad_w.append((1 - beta) * list_grad_w[i] ** 2)
            list_u_grad_b.append((1 - beta) * list_grad_b[i] ** 2)
        else :
          for i in range(len(list_grad_w)):
            list_u_grad_w.append(beta * prev_grad_uw[i] + (1 - beta) * list_grad_w[i] ** 2)
            list_u_grad_b.append(beta * prev_grad_ub[i] + (1 - beta) * list_grad_b[i] ** 2)
        
        for i in range(0,len(list_grad_w)):
          theta[i] -= (eta * list_grad_w[i].transpose())/(list_u_grad_w[i].transpose() + 1e-6) ** 0.5
          b[i] -= (eta * list_grad_b[i].transpose())/(list_u_grad_b[i].transpose()+ 1e-6) ** 0.5
        
        prev_grad_uw = list_u_grad_w.copy()
        prev_grad_ub = list_u_grad_b.copy()
        list_u_grad_w.clear()
        list_u_grad_b.clear()
        list_grad_w.clear()
        list_grad_b.clear()
      image_no += 1

    if (len(x_train) % batch_size != 0):
      list_grad_w.reverse()
      list_grad_b.reverse()
      for j in range(len(list_grad_w)):
          list_grad_w[j] += weight_decay * theta[j].transpose();
      if(len(prev_grad_uw) == 0):
        for i in range(len(list_grad_w)):
          prev_grad_uw.append(beta * list_grad_w[i])
          prev_grad_ub.append(beta * list_grad_b[i])

      for i in range(len(list_grad_w)):
        list_u_grad_w.append(beta * prev_grad_uw[i] + (1 - beta) * list_grad_w[i] ** 2)
        list_u_grad_b.append(beta * prev_grad_ub[i] + (1 - beta) * list_grad_b[i] ** 2)
      
      for i in range(0,len(list_grad_w)):
        theta[i] -= (eta * list_grad_w[i].transpose())/(list_u_grad_w[i].transpose() + 1e-6) ** 0.5
        b[i] -= (eta * list_grad_b[i].transpose())/(list_u_grad_b[i].transpose()+ 1e-6) ** 0.5
      
      prev_grad_uw = list_u_grad_w.copy()
      prev_grad_ub = list_u_grad_b.copy()
      list_u_grad_w.clear()
      list_u_grad_b.clear()
      list_grad_w.clear()
      list_grad_b.clear()

    for i in range(len(theta)):
        temp = (weight_decay / 2) * np.sum(np.square(theta[i]))
        loss += temp / len(x_train)
        val_loss += temp / len(x_val)
    print("training loss = ",loss)
    print("validation loss = ",val_loss)
    train_accuracy = image_pridiction(x_train,y_train,theta,b,num_layers,activation)
    print("trianing accuracy = ",train_accuracy)
    val_accuracy = image_pridiction(x_val,y_val,theta,b,num_layers,activation)
    print("validation_accuracy = ",val_accuracy)
    wandb.log({"train_accuracy": train_accuracy, "validation_accuracy": val_accuracy, "training_loss": loss, "validation cost": val_loss, 'epoch': t})
  return(theta,b)


In [ ]:
def adam_SGD(theta,b,epochs,eta,beta1,beta2,activation,x_train,y_train,neurons, num_layers,batch_size,loss_fun,weight_decay):
  
  t=0
  global x_val
  global y_val
  while t < epochs :
    t += 1
    print(t)
    image_no = 0
    list_u_grad_w = []
    list_u_grad_b = []
    list_grad_w = []
    list_grad_b = []
    list_m_w = [] # storing previous point
    list_m_b = [] # storing previous point
    list_v_w = [] # storing previous point
    list_v_b = [] # storing previous point
    loss = 0
    val_loss = 0
    while( image_no < len(x_train)):
      # forward propgation
      h=x_train[image_no].reshape(784,1)
      fp_list = forward_prop(activation,theta,b,num_layers,h)
      # backward propogation
      a_list = fp_list[2]
      h_list = fp_list[1]
      y_hat = fp_list[0]
      y = y_train[image_no]
      if( y_hat[y]!= 0):
        loss += loss_function(loss_fun,y_hat,y) / len(x_train)
      # validation loss
      if(image_no < len(x_val)):
        h_v = x_val[image_no].reshape(784,1)
        y_hat_v,h_list_v,a_list_v = forward_prop(activation,theta,b,num_layers,h_v)
        y_v = y_val[image_no]
        if(y_hat_v[y_v] != 0):
          val_loss += loss_function(loss_fun,y_hat_v,y_v) / len(x_val)

      # back propogation
      bp_list = backward_prop(theta,h_list,a_list,y_train[image_no],y_hat,num_layers ,batch_size,activation,loss_fun)
      list_grad_w=add_list_grad(list_grad_w.copy(), bp_list[0])
      list_grad_b=add_list_grad(list_grad_b.copy() , bp_list[1])
      
      if((image_no + 1) % batch_size == 0):
        list_grad_w.reverse()
        list_grad_b.reverse()
        for j in range(len(list_grad_w)):
          list_grad_w[j] += weight_decay * theta[j].transpose();
        if(len(list_m_w) == 0):
          for i in range(len(list_grad_w)):
            list_m_w.append((1 - beta1) * list_grad_w[i] )
            list_m_b.append((1 - beta1) * list_grad_b[i] )
        else :
          for i in range(len(list_grad_w)):
            list_m_w[i] = beta1 * list_m_w[i] + (1 - beta1) * list_grad_w[i] 
            list_m_b[i] = beta1 * list_m_b[i] + (1 - beta1) * list_grad_b[i] 
        
        if(len(list_v_w) == 0):
          for i in range(len(list_grad_w)):
            list_v_w.append((1 - beta2) * list_grad_w[i] ** 2 )
            list_v_b.append((1 - beta2) * list_grad_b[i] ** 2 )
        else :
          for i in range(len(list_grad_w)):
            list_v_w[i] = beta2 * list_v_w[i] + (1 - beta2) * list_grad_w[i] ** 2
            list_v_b[i] = beta2 * list_v_b[i] + (1 - beta2) * list_grad_b[i] ** 2
        
        list_mhat_w = []
        list_mhat_b = []
        list_vhat_w = []
        list_vhat_b = []
        for i in range(len(list_m_w)):
          list_mhat_w.append(list_m_w[i]/(1-beta1 ** (int(image_no /batch_size)+1)))
          list_mhat_b.append(list_m_b[i]/(1-beta1 ** (int(image_no /batch_size)+1)))
          list_vhat_w.append(list_v_w[i]/(1-beta2 ** (int(image_no /batch_size)+1)))
          list_vhat_b.append(list_v_b[i]/(1-beta2 ** (int(image_no /batch_size)+1)))

        
        # print(theta)
        for i in range(0,len(list_grad_w)):
          theta[i] -= (eta * list_mhat_w[i].transpose())/(list_vhat_w[i].transpose() + 1e-6) ** 0.5
          b[i] -= (eta * list_mhat_b[i].transpose())/(list_vhat_b[i].transpose()+ 1e-6) ** 0.5
        
        list_mhat_w.clear()
        list_mhat_b.clear()
        list_vhat_w.clear()
        list_vhat_b.clear()
        list_grad_w.clear()
        list_grad_b.clear()
      image_no += 1

    if (len(x_train) % batch_size != 0):
      list_grad_w.reverse()
      list_grad_b.reverse()
      for j in range(len(list_grad_w)):
          list_grad_w[j] += weight_decay * theta[j].transpose();
      if(len(list_m_w) == 0):
        for i in range(len(list_grad_w)):
          list_m_w.append((1 - beta1) * list_grad_w[i] )
          list_m_b.append((1 - beta1) * list_grad_b[i] )
      else :
        for i in range(len(list_grad_w)):
          list_m_w[i] = beta1 * list_m_w[i] + (1 - beta1) * list_grad_w[i] 
          list_m_b[i] = beta1 * list_m_b[i] + (1 - beta1) * list_grad_b[i] 
      
      if(len(list_v_w) == 0):
        for i in range(len(list_grad_w)):
          list_v_w.append((1 - beta2) * list_grad_w[i] ** 2 )
          list_v_b.append((1 - beta2) * list_grad_b[i] ** 2 )
      else :
        for i in range(len(list_grad_w)):
          list_v_w[i] = beta2 * list_v_w[i] + (1 - beta2) * list_grad_w[i] ** 2
          list_v_b[i] = beta2 * list_v_b[i] + (1 - beta2) * list_grad_b[i] ** 2
      
      list_mhat_w = []
      list_mhat_b = []
      list_vhat_w = []
      list_vhat_b = []
      for i in range(len(list_m_w)):
        list_mhat_w.append(list_m_w[i]/(1-beta1 ** int(image_no /batch_size)))
        list_mhat_b.append(list_m_b[i]/(1-beta1 ** int(image_no /batch_size)))
        list_vhat_w.append(list_v_w[i]/(1-beta2 ** int(image_no /batch_size)))
        list_vhat_b.append(list_v_b[i]/(1-beta2 ** int(image_no /batch_size)))

      
      # print(theta)
      for i in range(0,len(list_grad_w)):
        theta[i] -= (eta * list_mhat_w[i].transpose())/(list_vhat_w[i].transpose() + 1e-6) ** 0.5
        b[i] -= (eta * list_mhat_b[i].transpose())/(list_vhat_b[i].transpose()+ 1e-6) ** 0.5
      
      list_mhat_w.clear()
      list_mhat_b.clear()
      list_vhat_w.clear()
      list_vhat_b.clear()
      list_grad_w.clear()
      list_grad_b.clear()
    for i in range(len(theta)):
        temp = (weight_decay / 2) * np.sum(np.square(theta[i]))
        loss += temp / len(x_train)
        val_loss += temp / len(x_val)
    print("training loss = ",loss)
    print("validation loss = ",val_loss)
    train_accuracy = image_pridiction(x_train,y_train,theta,b,num_layers,activation)
    print("trianing accuracy = ",train_accuracy)
    val_accuracy = image_pridiction(x_val,y_val,theta,b,num_layers,activation)
    print("validation_accuracy = ",val_accuracy)
    wandb.log({"train_accuracy": train_accuracy, "validation_accuracy": val_accuracy, "training_loss": loss, "validation cost": val_loss, 'epoch': t})
  return(theta,b)


In [ ]:
def NAG(theta,b,epochs,eta,beta,activation,x_train,y_train,neurons, num_layers,batch_size,loss_fun,weight_decay):
  
  t=0
  global x_val
  global y_val
  while t < epochs :
    t += 1
    print(t)
    image_no = 0
    list_u_grad_w = []
    list_u_grad_b = []
    list_grad_w = []
    list_grad_b = []
    prev_grad_uw = [] # storing previous point
    prev_grad_ub = [] # storing previous point
    loss = 0
    val_loss = 0
    while(image_no<len(x_train)):
      # forward propgation
      h=x_train[image_no].reshape(784,1)
      w_lookahead = []
      b_lookahead = []
      if(len(prev_grad_uw)==0):
        w_lookahead = theta.copy()
        b_lookahead = b.copy();
      else :
        for i in range(len(prev_grad_uw)):
          w_lookahead.append(theta[i] - beta * prev_grad_uw[i].transpose())
          b_lookahead.append(b[i] - beta * prev_grad_ub[i].transpose())
      
      fp_list = forward_prop(activation,w_lookahead,b_lookahead,num_layers,h)
      # backward propogation
      a_list = fp_list[2]
      h_list = fp_list[1]
      y_hat = fp_list[0]
      y = y_train[image_no]
      if( y_hat[y]!= 0):
        loss += loss_function(loss_fun,y_hat,y) / len(x_train) 
      # validation loss
      if(image_no < len(x_val)):
        h_v = x_val[image_no].reshape(784,1)
        y_hat_v,h_list_v,a_list_v = forward_prop(activation,theta,b,num_layers,h_v)
        y_v = y_val[image_no]
        if(y_hat_v[y_v] != 0):
          val_loss += loss_function(loss_fun,y_hat_v,y_v) / len(x_val)

      # back propogation
      bp_list = backward_prop(w_lookahead,h_list,a_list,y_train[image_no],y_hat,num_layers ,batch_size,activation,loss_fun)
      list_grad_w = add_list_grad(list_grad_w.copy() , bp_list[0])
      list_grad_b = add_list_grad(list_grad_b.copy() , bp_list[1])
      w_lookahead.clear()
      b_lookahead.clear()
      if((image_no + 1) % batch_size == 0):
        list_grad_w.reverse()
        list_grad_b.reverse()
        for j in range(len(list_grad_w)):
          list_grad_w[j] += weight_decay * theta[j].transpose();
        if(len(prev_grad_uw) == 0):
          for i in range(len(list_grad_w)):
            list_u_grad_w = list_grad_w.copy()
            list_u_grad_b = list_grad_b.copy()
        else :
          for i in range(len(list_grad_w)):
            list_u_grad_w.append(beta * prev_grad_uw[i] + (1 - beta) * list_grad_w[i])
            list_u_grad_b.append(beta * prev_grad_ub[i] + (1 - beta) * list_grad_b[i])
        
        # print(theta)
        theta = add_list_grad_eta(theta.copy() , list_u_grad_w, eta)
        b = add_list_grad_eta(b.copy(), list_u_grad_b , eta)
        
        prev_grad_uw = list_u_grad_w.copy()
        prev_grad_ub = list_u_grad_b.copy()
        list_u_grad_w.clear()
        list_u_grad_b.clear()
        list_grad_w.clear()
        list_grad_b.clear()
        
      image_no += 1

    if (len(x_train) % batch_size != 0):
      list_grad_w.reverse()
      list_grad_b.reverse()
      for j in range(len(list_grad_w)):
          list_grad_w[j] += weight_decay * theta[j].transpose();
      if(len(prev_grad_uw) == 0):
        for i in range(len(list_grad_w)):
          prev_grad_uw.append(beta * list_grad_w[i])
          prev_grad_ub.append(beta * list_grad_b[i])

      for i in range(len(list_grad_w)):
        list_u_grad_w.append(beta * prev_grad_uw[i] + (1 - beta) * list_grad_w[i])
        list_u_grad_b.append(beta * prev_grad_ub[i] + (1 - beta) * list_grad_b[i])
      
      theta = add_list_grad_eta(theta.copy() , list_u_grad_w, eta)
      b = add_list_grad_eta(b.copy(), list_u_grad_b , eta)
      
      prev_grad_uw = list_u_grad_w.copy()
      prev_grad_ub = list_u_grad_b.copy()
      list_u_grad_w.clear()
      list_u_grad_b.clear()
      list_grad_w.clear()
      list_grad_b.clear()
    for i in range(len(theta)):
        temp = (weight_decay / 2) * np.sum(np.square(theta[i]))
        loss += temp / len(x_train)
        val_loss += temp / len(x_val)
    print("training loss = ",loss)
    print("validation loss = ",val_loss)
    train_accuracy = image_pridiction(x_train,y_train,theta,b,num_layers,activation)
    print("trianing accuracy = ",train_accuracy)
    val_accuracy = image_pridiction(x_val,y_val,theta,b,num_layers,activation)
    print("validation_accuracy = ",val_accuracy)
    wandb.log({"train_accuracy": train_accuracy, "validation_accuracy": val_accuracy, "training_loss": loss, "validation cost": val_loss, 'epoch': t})
  return(theta,b)


In [ ]:
def nadam(theta,b,epochs,eta,beta1,beta2,activation,x_train,y_train,neurons, num_layers,batch_size,loss_fun,weight_decay):

  t=0
  global x_val
  global y_val
  while t < epochs :
    t += 1
    print(t)
    image_no = 0
    list_u_grad_w = []
    list_u_grad_b = []
    list_grad_w = []
    list_grad_b = []
    list_m_w = [] # storing previous point
    list_m_b = [] # storing previous point
    list_v_w = [] # storing previous point
    list_v_b = [] # storing previous point
    loss = 0
    val_loss = 0
    while(image_no<len(x_train)):
      # forward propgation
      h=x_train[image_no].reshape(784,1)
      fp_list = forward_prop(activation,theta,b,num_layers,h)

      # backward propogation
      a_list = fp_list[2]
      h_list = fp_list[1]
      y_hat = fp_list[0]
      y = y_train[image_no]
      if( y_hat[y]!= 0):
        loss += loss_function(loss_fun,y_hat,y) / len(x_train)
      # validation loss
      if(image_no < len(x_val)):
        h_v = x_val[image_no].reshape(784,1)
        y_hat_v,h_list_v,a_list_v = forward_prop(activation,theta,b,num_layers,h_v)
        y_v = y_val[image_no]
        if(y_hat_v[y_v] != 0):
          val_loss += loss_function(loss_fun,y_hat_v,y_v) / len(x_val)

      # back propogation
      bp_list = backward_prop(theta,h_list,a_list,y_train[image_no],y_hat,num_layers ,batch_size,activation,loss_fun)
      list_grad_w=add_list_grad(list_grad_w.copy(), bp_list[0])
      list_grad_b=add_list_grad(list_grad_b.copy() , bp_list[1])
      
      if((image_no + 1) % batch_size == 0):
        list_grad_w.reverse()
        list_grad_b.reverse()
        for j in range(len(list_grad_w)):
          list_grad_w[j] += weight_decay * theta[j].transpose();
        if(len(list_m_w) == 0):
          for i in range(len(list_grad_w)):
            list_m_w.append((1 - beta1) * list_grad_w[i] )
            list_m_b.append((1 - beta1) * list_grad_b[i] )
        else :
          for i in range(len(list_grad_w)):
            list_m_w[i] = beta1 * list_m_w[i] + (1 - beta1) * list_grad_w[i] 
            list_m_b[i] = beta1 * list_m_b[i] + (1 - beta1) * list_grad_b[i] 
        
        if(len(list_v_w) == 0):
          for i in range(len(list_grad_w)):
            list_v_w.append((1 - beta2) * list_grad_w[i] ** 2 )
            list_v_b.append((1 - beta2) * list_grad_b[i] ** 2 )
        else :
          for i in range(len(list_grad_w)):
            list_v_w[i] = beta2 * list_v_w[i] + (1 - beta2) * list_grad_w[i] ** 2
            list_v_b[i] = beta2 * list_v_b[i] + (1 - beta2) * list_grad_b[i] ** 2
        
        list_mhat_w = []
        list_mhat_b = []
        list_vhat_w = []
        list_vhat_b = []
        for i in range(len(list_m_w)):
          list_mhat_w.append(list_m_w[i]/(1-beta1 ** (int(image_no /batch_size)+1)))
          list_mhat_b.append(list_m_b[i]/(1-beta1 ** (int(image_no /batch_size)+1)))
          list_vhat_w.append(list_v_w[i]/(1-beta2 ** (int(image_no /batch_size)+1)))
          list_vhat_b.append(list_v_b[i]/(1-beta2 ** (int(image_no /batch_size)+1)))

        
        for i in range(0,len(list_grad_w)):
          theta[i] -= (eta * list_mhat_w[i].transpose())/(list_vhat_w[i].transpose() + 1e-6) ** 0.5
          b[i] -= (eta * list_mhat_b[i].transpose())/(list_vhat_b[i].transpose()+ 1e-6) ** 0.5
        
        list_mhat_w.clear()
        list_mhat_b.clear()
        list_vhat_w.clear()
        list_vhat_b.clear()
        list_grad_w.clear()
        list_grad_b.clear()
      image_no += 1

    if (len(x_train) % batch_size != 0):
      list_grad_w.reverse()
      list_grad_b.reverse()
      for j in range(len(list_grad_w)):
          list_grad_w[j] += weight_decay * theta[j].transpose();
      if(len(list_m_w) == 0):
        for i in range(len(list_grad_w)):
          list_m_w.append((1 - beta1) * list_grad_w[i] )
          list_m_b.append((1 - beta1) * list_grad_b[i] )
      else :
        for i in range(len(list_grad_w)):
          list_m_w[i] = beta1 * list_m_w[i] + (1 - beta1) * list_grad_w[i] 
          list_m_b[i] = beta1 * list_m_b[i] + (1 - beta1) * list_grad_b[i] 
      
      if(len(list_v_w) == 0):
        for i in range(len(list_grad_w)):
          list_v_w.append((1 - beta2) * list_grad_w[i] ** 2 )
          list_v_b.append((1 - beta2) * list_grad_b[i] ** 2 )
      else :
        for i in range(len(list_grad_w)):
          list_v_w[i] = beta2 * list_v_w[i] + (1 - beta2) * list_grad_w[i] ** 2
          list_v_b[i] = beta2 * list_v_b[i] + (1 - beta2) * list_grad_b[i] ** 2
      
      list_mhat_w = []
      list_mhat_b = []
      list_vhat_w = []
      list_vhat_b = []
      for i in range(len(list_m_w)):
        list_mhat_w.append(list_m_w[i]/(1-beta1 ** int(image_no /batch_size)))
        list_mhat_b.append(list_m_b[i]/(1-beta1 ** int(image_no /batch_size)))
        list_vhat_w.append(list_v_w[i]/(1-beta2 ** int(image_no /batch_size)))
        list_vhat_b.append(list_v_b[i]/(1-beta2 ** int(image_no /batch_size)))

      nadam_term_w = []
      nadam_term_b = []
      for i in range(len(list_grad_w)):
        nadam_term_w.append( beta1 * list_mhat_w[i] + ( (1 - beta1) * list_grad_w[i]) / (1 - beta1 ** int(image_no /batch_size) ))
        nadam_term_b.append( beta1 * list_mhat_b[i] + ( (1 - beta1) * list_grad_b[i]) / (1 - beta1 ** int(image_no /batch_size) ))
      # print(theta)
      for i in range(0,len(list_grad_w)):
        theta[i] -= (eta * nadam_term_w[i].transpose())/(list_vhat_w[i].transpose() + 1e-6) ** 0.5
        b[i] -= (eta * nadam_term_b[i].transpose())/(list_vhat_b[i].transpose()+ 1e-6) ** 0.5
      
      nadam_term_w.clear()
      nadam_term_b.clear()
      list_mhat_w.clear()
      list_mhat_b.clear()
      list_vhat_w.clear()
      list_vhat_b.clear()
      list_grad_w.clear()
      list_grad_b.clear()
    for i in range(len(theta)):
        temp = (weight_decay / 2) * np.sum(np.square(theta[i]))
        loss += temp / len(x_train)
        val_loss += temp / len(x_val)
    print("training loss = ",loss)
    print("validation loss = ",val_loss)
    train_accuracy = image_pridiction(x_train,y_train,theta,b,num_layers,activation)
    print("trianing accuracy = ",train_accuracy)
    val_accuracy = image_pridiction(x_val,y_val,theta,b,num_layers,activation)
    print("validation_accuracy = ",val_accuracy)
    wandb.log({"train_accuracy": train_accuracy, "validation_accuracy": val_accuracy, "training_loss": loss, "validation cost": val_loss, 'epoch': t})
  return(theta,b)


In [ ]:
def neural_network(x_train,y_train,x_test,y_test,eta,beta1,beta2,activation,initializer,optimizer,batch_size,epochs,loss_fun,weight_decay,num_layers,neurons):
  neurons = [neurons]*(num_layers+2)
  neurons[0] = 784
  neurons[-1] = 10
  if(initializer == "xavier"):
    theta = initilize_theta_xavier(num_layers,neurons)
    b = initilize_bias(num_layers,neurons)
  elif (initializer == "random_normal"):
    theta = initilize_theta_random_normal(num_layers,neurons)
    b = initilize_bias(num_layers,neurons)
  elif (initializer == "random_uniform"):
    theta = initilize_theta_random_uniform(num_layers,neurons)
    b = initilize_bias(num_layers,neurons)
  else :
    print("Invalid Intilizer")
  
  # invoking training fun
  if(optimizer == "sgd"):
    theta,b = SGD(theta,b,epochs,eta,beta1,activation,x_train,y_train,neurons, num_layers,batch_size,loss_fun,weight_decay)
  elif(optimizer == "momentum"):
    theta,b = momentum_SGD(theta,b,epochs,eta,beta1,activation,x_train,y_train,neurons, num_layers,batch_size,loss_fun,weight_decay)
  elif(optimizer == "rms"):
    theta,b = RMS_SGD(theta,b,epochs,eta,beta1,activation,x_train,y_train,neurons, num_layers,batch_size,loss_fun,weight_decay)
  elif(optimizer == "adam"):
    theta,b = adam_SGD(theta,b,epochs,eta,beta1,beta2,activation,x_train,y_train,neurons, num_layers,batch_size,loss_fun,weight_decay)
  elif(optimizer == "nag"):
    theta,b = NAG(theta,b,epochs,eta,beta1,activation,x_train,y_train,neurons, num_layers,batch_size,loss_fun,weight_decay)
  elif(optimizer == "nadam"):
    theta,b = nadam(theta,b,epochs,eta,beta1,beta2,activation,x_train,y_train,neurons, num_layers,batch_size,loss_fun,weight_decay)
  else :
    print("Invalid Optimizer")


  return(theta,b)
 


In [ ]:
def run_NN():
    

    # Initialize a new wandb run
    wandb.init()
    
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config

    # Local variables, values obtained from wandb config
    num_neurons = config.num_neurons
    num_hidden = config.num_hidden
    init_mode = config.init_mode
    epochs = config.epochs
    batch_size = config.batch_size
    learning_rate = config.learning_rate
    activation_f = config.activation_f
    L2_lamb = config.L2_lamb
    optimizer = config.optimizer

    # Display the hyperparameters
    run_name = "lr_{}_ac_{}_in_{}_op_{}_bs_{}_L2_{}_ep_{}_nn_{}_nh_{}".format(learning_rate, activation_f, init_mode, optimizer, batch_size, L2_lamb, epochs, num_neurons, num_hidden)
    print(run_name)
    neural_network(x_train,y_train,x_test,y_test,learning_rate,0.9,0.999,activation_f,init_mode,optimizer,batch_size,epochs,"mse",0.0005,num_hidden,num_neurons)
    wandb.run.name = run_name
    wandb.run.save()

    

In [ ]:
sweep_config = {
  "name": "CS6910 Assignment 1 - Squared Error Loss",
  "metric": {
      "name":"validation_accuracy",
      "goal": "maximize"
  },
  "method": "random",
  "parameters": {
        "learning_rate": {
            "values": [0.1, 0.01]
        },
        "activation_f": {
            "values": ["sigmoid", "relu", "tanh"]
        },
        "init_mode": {
            "values": ["xavier", "random_normal"]
        },
        "optimizer": {
            "values": ["sgd", "momentum", "nag", "adam", "nadam", "rms"]
        },
        "batch_size": {
            "values": [16,32,64]
        },
        "epochs": {
            "values": [5]
        },
        "L2_lamb": {
            "values": [0, 0.0005, 0.5]
        },
        "num_neurons": {
            "values": [32,64,128]
        },
        "num_hidden": {
            "values": [2,3,4]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, entity="rajmahajan24", project="DL_A1_optimizers_combined_13_01(mse)")
wandb.agent(sweep_id, run_NN, count=120)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: zs5wny9u
Sweep URL: https://wandb.ai/rajmahajan24/DL_A1_optimizers_combined_13_01%28mse%29/sweeps/zs5wny9u


Exception in thread NetStatThr:
Traceback (most recent call last):
  File "/usr/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.9/dist-packages/wandb/sdk/wandb_run.py", line 258, in check_network_status
    self._loop_check_status(
  File "/usr/local/lib/python3.9/dist-packages/wandb/sdk/wandb_run.py", line 214, in _loop_check_status
    local_handle = request()
  File "/usr/local/lib/python3.9/dist-packages/wandb/sdk/interface/interface.py", line 795, in deliver_network_status
    return self._deliver_network_status(status)
  File "/usr/local/lib/python3.9/dist-packages/wandb/sdk/interface/interface_shared.py", line 601, in _deliver_network_status
    return self._deliver_record(record)
  File "/usr/local/lib/python3.9/dist-packages/wandb/sdk/interface/interface_shared.py", line 560, in _deliver_record
    handle = mailbox._d

lr_0.01_ac_relu_in_random_normal_op_adam_bs_32_L2_0_ep_5_nn_64_nh_2
1
training loss =  0.2818688300925255
validation loss =  0.4183552244436207
trianing accuracy =  9.964814814814815
validation_accuracy =  10.316666666666666
2
training loss =  0.5295248268806065
validation loss =  0.7186153042152723
trianing accuracy =  10.037037037037036
validation_accuracy =  9.666666666666666
3
training loss =  0.6750144606105285
validation loss =  0.7712773511636248
trianing accuracy =  10.005555555555556
validation_accuracy =  9.95
4
training loss =  0.7249056064466013
validation loss =  0.7111098726904982
trianing accuracy =  10.037037037037036
validation_accuracy =  9.666666666666666
5
training loss =  0.5013144731432437
validation loss =  0.7742752707967943
trianing accuracy =  9.898148148148149


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


...

wandb: Agent Starting Run: ppn3crdh with config:
wandb: 	L2_lamb: 0
wandb: 	activation_f: sigmoid
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	init_mode: random_normal
wandb: 	learning_rate: 0.01
wandb: 	num_hidden: 3
wandb: 	num_neurons: 64
wandb: 	optimizer: adam


lr_0.01_ac_sigmoid_in_random_normal_op_adam_bs_32_L2_0_ep_5_nn_64_nh_3
1
training loss =  0.20294032691450825
validation loss =  0.266924157541263
trianing accuracy =  71.46296296296296
validation_accuracy =  71.9
2
training loss =  0.21271970587756825
validation loss =  0.2092393519493381
trianing accuracy =  69.61666666666666
validation_accuracy =  69.1
3
training loss =  0.21423292005437006
validation loss =  0.20962414631690574
trianing accuracy =  67.04629629629629
validation_accuracy =  67.95
4
training loss =  0.2148328781770803
validation loss =  0.21093007658233334
trianing accuracy =  66.96481481481482
validation_accuracy =  67.85
5
training loss =  0.21581673530657242
validation loss =  0.21094035621177773
trianing accuracy =  70.14444444444445
validation_accuracy =  70.8


epoch,▁▃▅▆█
train_accuracy,█▅▁▁▆
training_loss,▁▆▇▇█
validation cost,█▁▁▁▁
validation_accuracy,█▃▁▁▆
epoch,5
train_accuracy,70.14444
training_loss,0.21582
validation cost,0.21094
validation_accuracy,70.8


wandb: Agent Starting Run: hra90z8z with config:
wandb: 	L2_lamb: 0.0005
wandb: 	activation_f: tanh
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	init_mode: xavier
wandb: 	learning_rate: 0.01
wandb: 	num_hidden: 4
wandb: 	num_neurons: 128
wandb: 	optimizer: adam


lr_0.01_ac_tanh_in_xavier_op_adam_bs_32_L2_0.0005_ep_5_nn_128_nh_4
1
training loss =  0.4230454487010719
validation loss =  0.28595977652140636
trianing accuracy =  10.005555555555556
validation_accuracy =  9.95
2
training loss =  0.4622705404020623
validation loss =  0.46177257640771296
trianing accuracy =  10.005555555555556
validation_accuracy =  9.95
3


In [ ]:
# wandb.log({"training_acc": train_acc, "validation_accuracy": val_acc, "training_loss": cost, "validation cost": val_cost, 'epoch': count})
